# Import Dependencies

In [ ]:
import pandas as pd
from sqlalchemy import create_engine  

# Bring in password for connection to SQL
from config import password

# Load CSV Data

In [ ]:
netflix_file = "Resources/netflix_titles.csv"
netflix_df = pd.read_csv(netflix_file)        
netflix_df.head()

In [ ]:
wiki_file = "Resources/wiki_movie_plots_deduped.csv"
wiki_df = pd.read_csv(wiki_file)
wiki_df.head()

# Transform titles_file

In [ ]:

# Create a filtered dataframe from specific columns
netflix_cols = ["title", "country", "release_year", "listed_in"]
netflix_transformed= netflix_df[netflix_cols].copy()

# Rename the column headers
netflix_transformed = netflix_transformed.rename(columns={"title": "title",
                                                          "country": "country",
                                                          "release_year": "release_year",
                                                            "listed_in" : "genre"})
netflix_transformed.head()

In [ ]:
# Filter only rows for "release_year" 2010 to 2018.

netflix_filtered = netflix_transformed[(netflix_transformed["release_year"] >= 2010) & (netflix_transformed["release_year"] < 2018)]
netflix_filtered

# Transform plot_file

In [ ]:
# Checked for missing Plot entries
wiki_df.dropna(subset=['Plot'], how='all', inplace=True)
#wiki_df


# Create a filtered dataframe from specific columns
wiki_cols = ["Title", "Release Year", "Director", "Genre", "Plot"]
wiki_transformed= wiki_df[wiki_cols].copy() 

# Rename the column headers
wiki_transformed = wiki_transformed.rename(columns={"Title": "title",
                                                    "Release Year" : "release_year",
                                                          "Director": "director",
                                                            "Genre" : "genre",
                                                           "Plot": "plot"})
                                           
wiki_transformed.head()

In [ ]:
wiki_filtered = wiki_transformed[(wiki_transformed["release_year"] >= 2010) & (wiki_transformed["release_year"] <= 2018)]
wiki_filtered

In [ ]:
wiki_filtered = wiki_filtered.drop_duplicates(subset=['title'], keep=False)

In [ ]:
#Connect to the local database

In [ ]:
rds_connection_string = (f"postgres:{password}@localhost:5432/movies_db")
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Connect to Engine
engine.table_names()

In [ ]:
#Use Pandas to plug in the Netflix CSV file data into the SQL database we created
netflix_filtered.to_sql(name='netflix', con=engine, if_exists='append',index=False)

In [ ]:
#Use Pandas to plug in the Wiki CSV file data into the SQL database we created
wiki_filtered.to_sql(name='wiki', con=engine, if_exists='append', index=False)

In [ ]:
#Confirm Netflix Data has been added by running a query
pd.read_sql_query('select * from netflix', con=engine).head()

In [ ]:
#Confirm Wiki Data has been added by running a query
pd.read_sql_query('select * from wiki', con=engine).head()